In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

In [2]:
#folder="https://raw.githubusercontent.com/hh26/EPL-Predictions/master/CSV/"
folder="CSV/"
fifaTR = pd.read_csv(folder+'FifaTeamRatings.csv')
finalDataset = pd.read_csv(folder+'final_dataset.csv')

## Team Ratings from FIFA index

In [3]:
fifaTR

,FIFA Season,Team,ATT,MID,DEF,OVR
0,5,Arsenal,92,91,89,90
1,5,Manchester United,91,90,85,88
2,5,Chelsea,87,88,86,87
3,5,Liverpool,88,86,82,84
4,5,Newcastle United,85,83,76,81
...,...,...,...,...,...,...
315,20,Aston Villa,77,75,75,76
316,20,Southampton,76,76,76,76
317,20,Brighton & Hove Albion,75,76,76,76
318,20,Sheffield United,73,77,75,75


In [4]:
finalDataset['Season'] = finalDataset.apply(lambda row: ((row.name)//380)+5, axis = 1)
del_cols=['FTR','HTP','ATP','HTWinStreak3','HTWinStreak5','HTLossStreak3','HTLossStreak5','ATWinStreak3','ATWinStreak5','ATLossStreak3','ATLossStreak5','HTGD','ATGD','DiffPts','DiffFormPts']
finalDataset.drop(del_cols, inplace=True, axis=1)

## Team Ratings Calculation and updation to match data

In [5]:
def getRating(season, team, ratType):   
    return int(fifaTR[(fifaTR['FIFA Season']==season) & (fifaTR['Team']==team)][ratType])

In [6]:
def getRatAttr(row, home, ratType):
    team=""
    if(home==True):
        team = row.HomeTeam
    else:
        team = row.AwayTeam
    
    r = getRating(row.Season, team, ratType)
    return r

In [7]:
finalDataset['HAttack'] = finalDataset.apply(lambda row: getRatAttr(row, True, 'ATT'), axis = 1)
finalDataset['AAttack'] = finalDataset.apply(lambda row: getRatAttr(row, False, 'ATT'), axis = 1)
finalDataset['HMidField'] = finalDataset.apply(lambda row: getRatAttr(row, True, 'MID'), axis = 1)
finalDataset['AMidField'] = finalDataset.apply(lambda row: getRatAttr(row, False, 'MID'), axis = 1)
finalDataset['HDefence'] = finalDataset.apply(lambda row: getRatAttr(row, True, 'DEF'), axis = 1)
finalDataset['ADefence'] = finalDataset.apply(lambda row: getRatAttr(row, False, 'DEF'), axis = 1)
finalDataset['HOverall'] = finalDataset.apply(lambda row: getRatAttr(row, True, 'OVR'), axis = 1)
finalDataset['AOverall'] = finalDataset.apply(lambda row: getRatAttr(row, False, 'OVR'), axis = 1)

## Match Info from Dataset-1

In [8]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'Season', 'HAttack', 'AAttack', 'HMidField', 'AMidField', 'HDefence', 'ADefence', 'HOverall', 'AOverall']
finalDataset[cols]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,Season,HAttack,AAttack,HMidField,AMidField,HDefence,ADefence,HOverall,AOverall
0,14-08-2004,Aston Villa,Southampton,2,0,5,79,80,74,72,77,73,77,75
1,14-08-2004,Blackburn Rovers,West Bromwich Albion,1,1,5,72,78,78,72,75,71,76,73
2,14-08-2004,Bolton Wanderers,Charlton Athletic,4,1,5,74,76,79,79,73,75,76,77
3,14-08-2004,Manchester City,Fulham,1,1,5,81,78,76,76,74,77,77,78
4,14-08-2004,Middlesbrough,Newcastle United,2,2,5,82,85,79,83,79,76,80,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,13-05-2018,Newcastle United,Chelsea,3,0,18,76,84,75,86,76,82,75,83
5316,13-05-2018,Southampton,Manchester City,0,1,18,76,85,77,87,75,83,76,84
5317,13-05-2018,Swansea City,Stoke City,1,2,18,76,75,75,76,74,76,75,76
5318,13-05-2018,Tottenham Hotspur,Leicester City,5,4,18,85,79,83,79,82,76,83,78


## Finding Result 

In [9]:
def findRes(FTHG,FTAG):
    if(FTHG>FTAG):
        return 'H'
    elif(FTHG<FTAG):
        return 'A'
    else:
        return 'D'

In [10]:
FTR=list(map(findRes,finalDataset.FTHG,finalDataset.FTAG))
finalDataset.insert(5,'FTR',FTR,True)

## Form Update

In [11]:
def updateForms(hteam, ateam, res, gamma, form_dict):
    if res=='H':
        form_dict[hteam]=form_dict[hteam]+gamma*form_dict[ateam]
        form_dict[ateam]=form_dict[ateam]-gamma*form_dict[ateam]
    elif res=='A':
        form_dict[ateam]=form_dict[ateam]+gamma*form_dict[hteam]
        form_dict[hteam]=form_dict[hteam]-gamma*form_dict[hteam]
    else:
        form_dict[hteam]=form_dict[hteam]-gamma*(form_dict[hteam] - form_dict[ateam])
        form_dict[ateam]=form_dict[ateam]-gamma*(form_dict[ateam] - form_dict[hteam])

In [12]:
s5 = pd.read_csv(folder+'2004-05.csv')
s6 = pd.read_csv(folder+'2005-06.csv')
s7 = pd.read_csv(folder+'2006-07.csv')
s8 = pd.read_csv(folder+'2007-08.csv')
s9 = pd.read_csv(folder+'2008-09.csv')
s10 = pd.read_csv(folder+'2009-10.csv')
s11 = pd.read_csv(folder+'2010-11.csv')
s12 = pd.read_csv(folder+'2011-12.csv')
s13 = pd.read_csv(folder+'2012-13.csv')
s14 = pd.read_csv(folder+'2013-14.csv')
s15 = pd.read_csv(folder+'2014-15.csv')
s16 = pd.read_csv(folder+'2015-16.csv')
s17 = pd.read_csv(folder+'2016-17.csv')
s18 = pd.read_csv(folder+'2017-18.csv')

seasonStatSets = [s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18]

In [13]:
def createFormColumns(seasonStatSets, gamma):   
    seasonNum=5
    for sdf in seasonStatSets:
        sdf['Season'] = seasonNum
        form_dict={}
        for t in sdf.HomeTeam.unique():
            form_dict[t]=1
        sdf['HForm']=1
        sdf['AForm']=1
        for index, row in sdf.iterrows(): 
            sdf.loc[index, "HForm"] = form_dict[row.HomeTeam]
            sdf.loc[index, "AForm"] = form_dict[row.AwayTeam]
            updateForms(row.HomeTeam, row.AwayTeam, row.FTR, gamma, form_dict)
        seasonNum+=1
        
createFormColumns(seasonStatSets, 0.33)

In [14]:
s7[['Date', 'HomeTeam', 'AwayTeam', 'Season', 'FTHG', 'FTAG', 'HForm', 'AForm']][0:40]

,Date,HomeTeam,AwayTeam,Season,FTHG,FTAG,HForm,AForm
0,19-08-2006,Arsenal,Aston Villa,7,1,1,1.000000,1.000000
1,19-08-2006,Bolton Wanderers,Tottenham Hotspur,7,2,0,1.000000,1.000000
2,19-08-2006,Everton,Watford,7,2,1,1.000000,1.000000
3,19-08-2006,Newcastle United,Wigan Athletic,7,2,1,1.000000,1.000000
4,19-08-2006,Portsmouth,Blackburn Rovers,7,3,0,1.000000,1.000000
5,19-08-2006,Reading,Middlesbrough,7,3,2,1.000000,1.000000
6,19-08-2006,Sheffield United,Liverpool,7,1,1,1.000000,1.000000
7,19-08-2006,West Ham United,Charlton Athletic,7,3,1,1.000000,1.000000
8,20-08-2006,Chelsea,Manchester City,7,3,0,1.000000,1.000000
9,20-08-2006,Manchester United,Fulham,7,5,1,1.000000,1.000000


## Last k goals

In [15]:
def createGKPPColumns(seasonStatSets, k):
    for sdf in seasonStatSets:
        kmo={}
    
        sdf.loc[:, 'HGKPP'] = 0
        sdf.loc[:, 'AGKPP'] = 0
    
        for n in sdf.HomeTeam.unique():
            kmo[n]=[]
    
        for index, row in sdf.iterrows(): 
            ht=row.HomeTeam
            at=row.AwayTeam
            sdf.loc[index, 'HGKPP'] = sum(kmo[ht])/k
            sdf.loc[index, 'AGKPP'] = sum(kmo[at])/k
            kmo[ht].append(row.FTHG)
            kmo[at].append(row.FTAG)
            if(len(kmo[ht])==k+1):
                kmo[ht].pop(0)
            if(len(kmo[at])==k+1):
                kmo[at].pop(0)      
                
createGKPPColumns(seasonStatSets, 6)

In [16]:
s5[200:280]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AC,HY,AY,HR,AR,Season,HForm,AForm,HGKPP,AGKPP
200,E0,01-01-2005,Aston Villa,Blackburn Rovers,1,0,H,0,0,D,...,7,1,0,0,0,5,0.261233,0.868922,0.333333,0.833333
201,E0,01-01-2005,Bolton Wanderers,West Bromwich Albion,1,1,D,0,1,A,...,3,0,1,0,0,5,0.132535,0.194244,0.666667,0.500000
202,E0,01-01-2005,Charlton Athletic,Arsenal,1,3,A,1,1,D,...,2,1,1,0,0,5,1.449801,2.192488,1.000000,1.666667
203,E0,01-01-2005,Fulham,Crystal Palace,3,1,H,1,1,D,...,2,1,3,0,0,5,0.293015,0.550680,0.833333,0.833333
204,E0,01-01-2005,Liverpool,Chelsea,0,1,A,0,0,D,...,2,2,2,0,0,5,1.358830,2.841323,1.833333,2.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,E0,28-02-2005,Norwich City,Manchester City,2,3,A,2,2,D,...,12,2,2,1,0,5,0.305202,1.059440,1.333333,0.833333
276,E0,05-03-2005,Arsenal,Portsmouth,3,0,H,1,0,H,...,3,0,2,0,0,5,1.183408,0.094906,2.000000,0.833333
277,E0,05-03-2005,Aston Villa,Middlesbrough,2,0,H,0,0,D,...,4,1,2,0,0,5,0.220805,0.839637,1.500000,1.500000
278,E0,05-03-2005,Crystal Palace,Manchester United,0,0,D,0,0,D,...,7,5,4,1,0,5,0.566389,3.987355,1.500000,2.333333


In [17]:
s5.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'Season', 'HForm', 'AForm', 'HGKPP',
       'AGKPP'],
      dtype='object')

## Past k SOT

In [18]:
def createSTKPPColumns(seasonStatSets, k):
    for sdf in seasonStatSets:
        kmo={}
    
        sdf.loc[:, 'HSTKPP'] = 0
        sdf.loc[:, 'ASTKPP'] = 0
    
        for n in sdf.HomeTeam.unique():
            kmo[n]=[]
    
        for index, row in sdf.iterrows(): 
            ht=row.HomeTeam
            at=row.AwayTeam
            sdf.loc[index, 'HSTKPP'] = sum(kmo[ht])/k
            sdf.loc[index, 'ASTKPP'] = sum(kmo[at])/k
            kmo[ht].append(row.HST)
            kmo[at].append(row.AST)
            if(len(kmo[ht])==k+1):
                kmo[ht].pop(0)
            if(len(kmo[at])==k+1):
                kmo[at].pop(0)   
                
createSTKPPColumns(seasonStatSets, 6)

In [19]:
cols=['HomeTeam', 'AwayTeam','HST', 'AST', 'HSTKPP', 'ASTKPP']
s5[cols][230:280]

,HomeTeam,AwayTeam,HST,AST,HSTKPP,ASTKPP
230,Birmingham City,Fulham,5,3,7.000000,4.833333
231,Chelsea,Portsmouth,9,5,7.666667,4.333333
232,Crystal Palace,Tottenham Hotspur,7,6,5.666667,9.333333
233,Everton,Charlton Athletic,7,3,4.333333,5.666667
234,Manchester United,Aston Villa,11,6,8.833333,6.333333
235,Norwich City,Middlesbrough,9,15,3.333333,6.666667
236,Southampton,Liverpool,7,2,3.666667,7.000000
237,West Bromwich Albion,Manchester City,6,5,4.666667,5.500000
238,Arsenal,Newcastle United,8,1,6.666667,6.500000
239,Blackburn Rovers,Bolton Wanderers,4,3,7.666667,6.000000


## Past k corners

In [20]:
def createCKPPColumns(seasonStatSets, k):
    for sdf in seasonStatSets:
        kmo={}
    
        sdf.loc[:, 'HCKPP'] = 0
        sdf.loc[:, 'ACKPP'] = 0
    
        for n in sdf.HomeTeam.unique():
            kmo[n]=[]
    
        for index, row in sdf.iterrows(): 
            ht=row.HomeTeam
            at=row.AwayTeam
            sdf.loc[index, 'HCKPP'] = sum(kmo[ht])/k
            sdf.loc[index, 'ACKPP'] = sum(kmo[at])/k
            kmo[ht].append(row.HC)
            kmo[at].append(row.AC)
            if(len(kmo[ht])==k+1):
                kmo[ht].pop(0)
            if(len(kmo[at])==k+1):
                kmo[at].pop(0)
                
createCKPPColumns(seasonStatSets, 6)

In [21]:
cols=['HomeTeam', 'AwayTeam','HC', 'AC', 'HCKPP', 'ACKPP']
s5[cols][230:280]

,HomeTeam,AwayTeam,HC,AC,HCKPP,ACKPP
230,Birmingham City,Fulham,5,3,4.166667,6.166667
231,Chelsea,Portsmouth,2,3,5.000000,5.000000
232,Crystal Palace,Tottenham Hotspur,5,4,5.666667,5.166667
233,Everton,Charlton Athletic,9,5,4.500000,6.166667
234,Manchester United,Aston Villa,4,2,8.166667,5.833333
235,Norwich City,Middlesbrough,5,8,5.333333,6.333333
236,Southampton,Liverpool,6,3,4.166667,6.166667
237,West Bromwich Albion,Manchester City,3,11,2.500000,6.166667
238,Arsenal,Newcastle United,11,1,7.166667,6.666667
239,Blackburn Rovers,Bolton Wanderers,7,7,5.666667,5.500000


## MRKPP, Streak and WeightedStreak

In [22]:
points={'W':3,'L':0,'D':1}

def weightCalc(pastRes, k):
    if('M' in pastRes):  
        return -1
    else:
        sum = 0
        for i in range(k):
            sum+=points[pastRes[i]]*(k-i)
        return sum*2/(3*k*(k+1))
    

def createMRKPP_St_StWeighted_Columns(seasonStatSets, k):
    for sdf in seasonStatSets:
        res_kmo={}
        pt_kmo={}
    
        sdf.loc[:, 'HMRKPP'] = ''
        sdf.loc[:, 'AMRKPP'] = ''
        sdf.loc[:, 'HSt'] = 0
        sdf.loc[:, 'ASt'] = 0
        sdf.loc[:, 'HStWeighted'] = 0
        sdf.loc[:, 'AStWeighted'] = 0    
        for n in sdf.HomeTeam.unique():
            res_kmo[n]=['M']*k
            pt_kmo[n]=[]
    
        for index, row in sdf.iterrows(): 
            ht=row.HomeTeam
            at=row.AwayTeam
            sdf.loc[index, 'HMRKPP'] = ''.join(res_kmo[ht])
            sdf.loc[index, 'AMRKPP'] = ''.join(res_kmo[at])
            sdf.loc[index, 'HSt'] = sum(pt_kmo[ht])/(3*k)
            sdf.loc[index, 'ASt'] = sum(pt_kmo[at])/(3*k)
            sdf.loc[index, 'HStWeighted'] = weightCalc(res_kmo[ht], k)
            sdf.loc[index, 'AStWeighted'] = weightCalc(res_kmo[at], k)   
            res_kmo[ht].pop()
            res_kmo[at].pop()
            if(row.FTHG > row.FTAG):
                res_kmo[ht].insert(0, 'W')
                pt_kmo[ht].insert(0, 3)
                res_kmo[at].insert(0, 'L')
                pt_kmo[at].insert(0, 0)
            elif(row.FTHG < row.FTAG):
                res_kmo[ht].insert(0, 'L')
                pt_kmo[ht].insert(0, 0)
                res_kmo[at].insert(0, 'W')
                pt_kmo[at].insert(0, 3)
            else:
                res_kmo[ht].insert(0, 'D')
                pt_kmo[ht].insert(0, 1)
                res_kmo[at].insert(0, 'D')
                pt_kmo[at].insert(0, 1)
                
            if(len(pt_kmo[ht])==k+1):
                pt_kmo[ht].pop()
            if(len(pt_kmo[at])==k+1):
                pt_kmo[at].pop()
                                
createMRKPP_St_StWeighted_Columns(seasonStatSets, 6)

In [23]:
s7[['Date', 'HomeTeam', 'AwayTeam', 'HMRKPP', 'AMRKPP', 'HSt', 'ASt', 'HStWeighted', 'AStWeighted']][50:70]

,Date,HomeTeam,AwayTeam,HMRKPP,AMRKPP,HSt,ASt,HStWeighted,AStWeighted
50,23-09-2006,Aston Villa,Charlton Athletic,DDWWDM,LLWLLM,0.500000,0.166667,-1.000000,-1.000000
51,23-09-2006,Fulham,Chelsea,DWWDLM,WWWLWM,0.444444,0.666667,-1.000000,-1.000000
52,23-09-2006,Liverpool,Tottenham Hotspur,WLLWDM,DLLWLM,0.388889,0.222222,-1.000000,-1.000000
53,23-09-2006,Manchester City,West Ham United,LLWDLM,LDLDWM,0.222222,0.277778,-1.000000,-1.000000
54,23-09-2006,Middlesbrough,Blackburn Rovers,DDLWLM,WDLDLM,0.277778,0.277778,-1.000000,-1.000000
55,23-09-2006,Reading,Manchester United,WWLLWM,LWWWWM,0.500000,0.666667,-1.000000,-1.000000
56,23-09-2006,Wigan Athletic,Watford,DLWLMM,DLLDLM,0.222222,0.111111,-1.000000,-1.000000
57,24-09-2006,Newcastle United,Everton,LWLLWM,DWWDWM,0.333333,0.611111,-1.000000,-1.000000
58,25-09-2006,Portsmouth,Bolton Wanderers,WWWDWM,DWLDWM,0.722222,0.444444,-1.000000,-1.000000
59,30-09-2006,Bolton Wanderers,Liverpool,WDWLDW,WWLLWD,0.611111,0.555556,0.634921,0.634921


## Combining seasons DF

In [24]:
cols=['Date', 'HomeTeam', 'AwayTeam', 'Season', 'HForm', 'AForm', 'HGKPP', 'AGKPP', 'HSTKPP', 'ASTKPP', 'HCKPP', 'ACKPP', 'HMRKPP', 'AMRKPP', 'HSt', 'ASt', 'HStWeighted', 'AStWeighted']
df2 = pd.concat([s5[cols], s6[cols], s7[cols], s8[cols], s9[cols], s10[cols], s11[cols], s12[cols]], ignore_index=True)
df2 = pd.concat([df2, s13[cols], s14[cols], s15[cols], s16[cols], s17[cols], s18[cols]], ignore_index=True)
df2

,Date,HomeTeam,AwayTeam,Season,HForm,AForm,HGKPP,AGKPP,HSTKPP,ASTKPP,HCKPP,ACKPP,HMRKPP,AMRKPP,HSt,ASt,HStWeighted,AStWeighted
0,14-08-2004,Aston Villa,Southampton,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MMMMMM,MMMMMM,0.000000,0.000000,-1.000000,-1.000000
1,14-08-2004,Blackburn Rovers,West Bromwich Albion,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MMMMMM,MMMMMM,0.000000,0.000000,-1.000000,-1.000000
2,14-08-2004,Bolton Wanderers,Charlton Athletic,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MMMMMM,MMMMMM,0.000000,0.000000,-1.000000,-1.000000
3,14-08-2004,Manchester City,Fulham,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MMMMMM,MMMMMM,0.000000,0.000000,-1.000000,-1.000000
4,14-08-2004,Middlesbrough,Newcastle United,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MMMMMM,MMMMMM,0.000000,0.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,13/05/2018,Newcastle United,Chelsea,18,0.321132,1.470010,0.833333,1.500000,2.666667,4.666667,4.500000,5.500000,LLLLWW,DWWWWD,0.333333,0.777778,0.142857,0.777778
5316,13/05/2018,Southampton,Manchester City,18,0.695583,3.649774,1.333333,2.833333,6.000000,6.666667,4.666667,6.833333,WDWDLL,WDWWWL,0.444444,0.722222,0.603175,0.793651
5317,13/05/2018,Swansea City,Stoke City,18,0.184760,0.524898,0.333333,0.666667,3.000000,3.166667,3.666667,3.000000,LLLLDD,LDDDLL,0.111111,0.166667,0.047619,0.190476
5318,13/05/2018,Tottenham Hotspur,Leicester City,18,1.373157,0.401427,1.166667,0.833333,4.666667,3.833333,5.166667,7.000000,WLWDLW,WLLDLL,0.555556,0.222222,0.571429,0.333333


# Goal Difference

In [25]:
finalDataset['HTGD']=finalDataset.HTGS-finalDataset.HTGC
finalDataset['ATGD']=finalDataset.ATGS-finalDataset.ATGC

In [26]:
cols=['Date', 'HomeTeam', 'AwayTeam', 'Season', 'HAttack', 'AAttack', 'HMidField', 'AMidField', 'HDefence', 'ADefence', 'HOverall', 'AOverall', 'HTGD', 'ATGD', 'FTR']
df1=finalDataset[cols]
df1

,Date,HomeTeam,AwayTeam,Season,HAttack,AAttack,HMidField,AMidField,HDefence,ADefence,HOverall,AOverall,HTGD,ATGD,FTR
0,14-08-2004,Aston Villa,Southampton,5,79,80,74,72,77,73,77,75,0,0,H
1,14-08-2004,Blackburn Rovers,West Bromwich Albion,5,72,78,78,72,75,71,76,73,0,0,D
2,14-08-2004,Bolton Wanderers,Charlton Athletic,5,74,76,79,79,73,75,76,77,0,0,H
3,14-08-2004,Manchester City,Fulham,5,81,78,76,76,74,77,77,78,0,0,D
4,14-08-2004,Middlesbrough,Newcastle United,5,82,85,79,83,79,76,80,81,0,0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,13-05-2018,Newcastle United,Chelsea,18,76,84,75,86,76,82,75,83,-11,27,H
5316,13-05-2018,Southampton,Manchester City,18,76,85,77,87,75,83,76,84,-18,78,A
5317,13-05-2018,Swansea City,Stoke City,18,76,75,75,76,74,76,75,76,-27,-34,A
5318,13-05-2018,Tottenham Hotspur,Leicester City,18,85,79,83,79,82,76,83,78,37,-3,H


# Combining all the required DF

In [27]:
finalDf = df2.merge(df1, on=['HomeTeam', 'AwayTeam', 'Season'])
finalDf

,Date_x,HomeTeam,AwayTeam,Season,HForm,AForm,HGKPP,AGKPP,HSTKPP,ASTKPP,...,AAttack,HMidField,AMidField,HDefence,ADefence,HOverall,AOverall,HTGD,ATGD,FTR
0,14-08-2004,Aston Villa,Southampton,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,80,74,72,77,73,77,75,0,0,H
1,14-08-2004,Blackburn Rovers,West Bromwich Albion,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,78,78,72,75,71,76,73,0,0,D
2,14-08-2004,Bolton Wanderers,Charlton Athletic,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,76,79,79,73,75,76,77,0,0,H
3,14-08-2004,Manchester City,Fulham,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,78,76,76,74,77,77,78,0,0,D
4,14-08-2004,Middlesbrough,Newcastle United,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,85,79,83,79,76,80,81,0,0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,13/05/2018,Newcastle United,Chelsea,18,0.321132,1.470010,0.833333,1.500000,2.666667,4.666667,...,84,75,86,76,82,75,83,-11,27,H
5316,13/05/2018,Southampton,Manchester City,18,0.695583,3.649774,1.333333,2.833333,6.000000,6.666667,...,85,77,87,75,83,76,84,-18,78,A
5317,13/05/2018,Swansea City,Stoke City,18,0.184760,0.524898,0.333333,0.666667,3.000000,3.166667,...,75,75,76,74,76,75,76,-27,-34,A
5318,13/05/2018,Tottenham Hotspur,Leicester City,18,1.373157,0.401427,1.166667,0.833333,4.666667,3.833333,...,79,83,79,82,76,83,78,37,-3,H


In [28]:
finalDf.columns

Index(['Date_x', 'HomeTeam', 'AwayTeam', 'Season', 'HForm', 'AForm', 'HGKPP',
       'AGKPP', 'HSTKPP', 'ASTKPP', 'HCKPP', 'ACKPP', 'HMRKPP', 'AMRKPP',
       'HSt', 'ASt', 'HStWeighted', 'AStWeighted', 'Date_y', 'HAttack',
       'AAttack', 'HMidField', 'AMidField', 'HDefence', 'ADefence', 'HOverall',
       'AOverall', 'HTGD', 'ATGD', 'FTR'],
      dtype='object')

In [29]:
colsInOrder = ['Season', 'HForm', 'AForm', 'HSt', 'ASt', 'HSTKPP', 'ASTKPP', 'HGKPP', 'AGKPP', 
                'HCKPP', 'ACKPP', 'HAttack', 'AAttack', 'HMidField', 'AMidField', 'HDefence', 
                'ADefence', 'HOverall', 'AOverall', 'HTGD', 'ATGD', 'HStWeighted', 'AStWeighted', 'FTR']
finalDf = finalDf[colsInOrder]

# Class A features DataFrame

In [30]:
finalDf

,Season,HForm,AForm,HSt,ASt,HSTKPP,ASTKPP,HGKPP,AGKPP,HCKPP,...,AMidField,HDefence,ADefence,HOverall,AOverall,HTGD,ATGD,HStWeighted,AStWeighted,FTR
0,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,72,77,73,77,75,0,0,-1.000000,-1.000000,H
1,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,72,75,71,76,73,0,0,-1.000000,-1.000000,D
2,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,79,73,75,76,77,0,0,-1.000000,-1.000000,H
3,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,76,74,77,77,78,0,0,-1.000000,-1.000000,D
4,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,83,79,76,80,81,0,0,-1.000000,-1.000000,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,18,0.321132,1.470010,0.333333,0.777778,2.666667,4.666667,0.833333,1.500000,4.500000,...,86,76,82,75,83,-11,27,0.142857,0.777778,H
5316,18,0.695583,3.649774,0.444444,0.722222,6.000000,6.666667,1.333333,2.833333,4.666667,...,87,75,83,76,84,-18,78,0.603175,0.793651,A
5317,18,0.184760,0.524898,0.111111,0.166667,3.000000,3.166667,0.333333,0.666667,3.666667,...,76,74,76,75,76,-27,-34,0.047619,0.190476,A
5318,18,1.373157,0.401427,0.555556,0.222222,4.666667,3.833333,1.166667,0.833333,5.166667,...,79,82,76,83,78,37,-3,0.571429,0.333333,H


## Differential Features

In [31]:
finalDf.loc[:, 'FormDifferential']= finalDf.HForm - finalDf.AForm
finalDf.loc[:, 'StDifferential']= finalDf.HSt - finalDf.ASt

finalDf.loc[:, 'STKPP']= finalDf.HSTKPP - finalDf.ASTKPP
finalDf.loc[:, 'GKPP']= finalDf.HGKPP - finalDf.AGKPP
finalDf.loc[:, 'CKPP']= finalDf.HCKPP - finalDf.ACKPP

finalDf.loc[:, 'RelAttack']= finalDf.HAttack - finalDf.AAttack
finalDf.loc[:, 'RelMidField']= finalDf.HMidField - finalDf.AMidField
finalDf.loc[:, 'RelDefence']= finalDf.HDefence - finalDf.ADefence
finalDf.loc[:, 'RelOverall']= finalDf.HOverall - finalDf.AOverall

finalDf.loc[:, 'GDDifferential']= finalDf.HTGD - finalDf.ATGD
finalDf.loc[:, 'StWeightedDifferential']= finalDf.HStWeighted - finalDf.AStWeighted

D:\Tools\Anaconda\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\Tools\Anaconda\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Class A and B included DataFrame

In [32]:
finalDf

,Season,HForm,AForm,HSt,ASt,HSTKPP,ASTKPP,HGKPP,AGKPP,HCKPP,...,StDifferential,STKPP,GKPP,CKPP,RelAttack,RelMidField,RelDefence,RelOverall,GDDifferential,StWeightedDifferential
0,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1,2,4,2,0,0.000000
1,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-6,6,4,3,0,0.000000
2,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-2,0,-2,-1,0,0.000000
3,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3,0,-3,-1,0,0.000000
4,5,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-3,-4,3,-1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,18,0.321132,1.470010,0.333333,0.777778,2.666667,4.666667,0.833333,1.500000,4.500000,...,-0.444444,-2.000000,-0.666667,-1.000000,-8,-11,-6,-8,-38,-0.634921
5316,18,0.695583,3.649774,0.444444,0.722222,6.000000,6.666667,1.333333,2.833333,4.666667,...,-0.277778,-0.666667,-1.500000,-2.166667,-9,-10,-8,-8,-96,-0.190476
5317,18,0.184760,0.524898,0.111111,0.166667,3.000000,3.166667,0.333333,0.666667,3.666667,...,-0.055556,-0.166667,-0.333333,0.666667,1,-1,-2,-1,7,-0.142857
5318,18,1.373157,0.401427,0.555556,0.222222,4.666667,3.833333,1.166667,0.833333,5.166667,...,0.333333,0.833333,0.333333,-1.833333,6,4,6,5,40,0.238095


In [33]:
cols=['FormDifferential','StDifferential','STKPP','GKPP','CKPP','RelAttack','RelMidField','RelDefence','RelOverall','GDDifferential','StWeightedDifferential']
finalDf[cols]

,FormDifferential,StDifferential,STKPP,GKPP,CKPP,RelAttack,RelMidField,RelDefence,RelOverall,GDDifferential,StWeightedDifferential
0,0.000000,0.000000,0.000000,0.000000,0.000000,-1,2,4,2,0,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,-6,6,4,3,0,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,-2,0,-2,-1,0,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,3,0,-3,-1,0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,-3,-4,3,-1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5315,-1.148878,-0.444444,-2.000000,-0.666667,-1.000000,-8,-11,-6,-8,-38,-0.634921
5316,-2.954190,-0.277778,-0.666667,-1.500000,-2.166667,-9,-10,-8,-8,-96,-0.190476
5317,-0.340139,-0.055556,-0.166667,-0.333333,0.666667,1,-1,-2,-1,7,-0.142857
5318,0.971730,0.333333,0.833333,0.333333,-1.833333,6,4,6,5,40,0.238095


# Pre-Processing

In [34]:
train = finalDf[(finalDf.Season<=14) & (finalDf.Season>5)]
test = finalDf[(finalDf.Season>14) & (finalDf.Season<=16)]

train = train[((train['HStWeighted']!=-1) & (train['AStWeighted']!=-1))]
test = test[((test['HStWeighted']!=-1) & (test['AStWeighted']!=-1))]

x_train = train.loc[:, (train.columns != 'FTR') & (train.columns != 'Season')]
y_train = train['FTR']

x_test = test.loc[:, (test.columns != 'FTR') & (test.columns != 'Season')]
y_test = test['FTR']


x_train.columns

Index(['HForm', 'AForm', 'HSt', 'ASt', 'HSTKPP', 'ASTKPP', 'HGKPP', 'AGKPP',
       'HCKPP', 'ACKPP', 'HAttack', 'AAttack', 'HMidField', 'AMidField',
       'HDefence', 'ADefence', 'HOverall', 'AOverall', 'HTGD', 'ATGD',
       'HStWeighted', 'AStWeighted', 'FormDifferential', 'StDifferential',
       'STKPP', 'GKPP', 'CKPP', 'RelAttack', 'RelMidField', 'RelDefence',
       'RelOverall', 'GDDifferential', 'StWeightedDifferential'],
      dtype='object')

In [35]:
y_trainDF = pd.DataFrame(data = y_train, columns = ['FTR'])
y_testDF = pd.DataFrame(data = y_test, columns = ['FTR'])

In [36]:
x_train.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\XTRAIN.csv', index = False)
x_test.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\XTEST.csv', index = False)
y_trainDF.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\YTRAIN.csv', index = False)
y_testDF.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\YTEST.csv', index = False)

train.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\TRAIN_CSV.csv', index = False)
test.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\TEST_CSV.csv', index = False)

In [37]:
test2 = finalDf[(finalDf.Season>16) & (finalDf.Season<=18)]
test2 = test2[((test2['HStWeighted']!=-1) & (test2['AStWeighted']!=-1))]
x_test2 = test2.loc[:, (test2.columns != 'FTR') & (test2.columns != 'Season')]
y_test2 = test2['FTR']
y_test2DF = pd.DataFrame(data = y_test2, columns = ['FTR'])
x_test2.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\XTEST2.csv', index = False)
y_test2DF.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\YTEST2.csv', index = False)

In [45]:
train2 = finalDf[(finalDf.Season<=16) & (finalDf.Season>5)]
train2 = train2[((train2['HStWeighted']!=-1) & (train2['AStWeighted']!=-1))]
x_train2 = train2.loc[:, (train2.columns != 'FTR') & (train2.columns != 'Season')]
y_train2 = train2['FTR']
y_train2DF = pd.DataFrame(data = y_train2, columns = ['FTR'])
x_train2.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\XTRAIN2.csv', index = False)
y_train2DF.to_csv(r'D:\College\SEM7\Journals\miniProjectWork\YTRAIN2.csv', index = False)